#Introduction to Microtonal Music Making through the Tonality Diamond

This notebook introduces several concepts and realizes music using those concepts
- Intro to the Tonality Diamond to the 31-limit

## Size of the Tonality Diamond, where n is the limit:

Wikipedia: https://en.wikipedia.org/wiki/Tonality_diamond 

According to wiki, there is a direct way to calculate how many unique pitches in a tonality diamond. <blockquote>
    If φ(n) is Euler's totient function, which gives the number of positive integers less than n and relatively prime to n, that is, it counts the integers less than n which share no common factor with n, and if d(n) denotes the size of the n-limit tonality diamond, we have the formula.
</blockquote>

They go on to describe Euler's Totient:
<img src='http://ripnread.com/listen/EulerTotientFunction.svg'>
Again according to wiki, there are 
- 7 members to the 5-limit diamond, 
- 13 to the 7-limit diamond, 
- 19 to the 9-limit diamond, 
- 29 to the 11-limit diamond, 
- 41 to the 13-limit diamond, and 
- 49 to the 15-limit diamond; 
And they further state that "that these suffice for most purposes."

I disagree, and have been using the tonality diamond to the 31 limit for about five years or so.

In [ ]:
import numpy as np
import numpy.testing as npt
import pprint
pp = pprint.PrettyPrinter(sort_dicts=False)
import copy
import mido
import time
from importlib import reload
from midi2audio import FluidSynth
from IPython.display import Audio, display
import music21 as m
import os
import muspy
import pandas as pd
import sys
sys.path.insert(0, '/home/prent/Dropbox/Tutorials/coconet-pytorch/coconet-pytorch-csound')
import piano as p
import selective_stretching_codes as stretch
import samples_used as su
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
soundfont = '../coconet-pytorch/font.sf2' # you will need to download this from location specified in the github README.md
midi_dir = 'eval'
CSD_FILE = 'ball3.csd'
LOGNAME = 'ball3.log'
WAV_OUT = '/home/prent/Music/sflib/ball3.wav'
CON_OUT = '/home/prent/Music/sflib/ball3-t'
downbeat = 1 # all the synthetic chorales out of TonicNet have a downbeat of 1
from fractions import Fraction
OCTAVE_SIZE = 213 # there are 213 distinct pictches per octave in the tonality diamond to the 31-limit
first_item_only = True

In [ ]:
# brute force method to calculate the size of a tonality diamond

for limit in (5, 7, 9, 11, 13, 15, 31):
    end_denom = limit + 1
    start_denom = (end_denom) // 2
    # print(f'{limit = }, {start_denom = }, {end_denom = }')
    o_numerator = np.arange(start_denom, end_denom,1)
    u_denominator = np.arange(start_denom, end_denom, 1)
    print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
    all_ratios = []
    for oton_root in u_denominator:
        print()
        for overtone in o_numerator:
            if overtone < oton_root: oton = overtone * 2
            else: oton = overtone
            print(f'{str(Fraction(oton / oton_root).limit_denominator())}', end = '\t')
            all_ratios.append(oton / oton_root)

    all_ratios = list(dict.fromkeys(all_ratios)) # eliminate duplicates
    all_ratios.sort()
    print(f'\nThere are {len(all_ratios)} unique members in the {limit}-limit diamond\n\n')      

In [ ]:
# We now have an list with all the ratios in the diamond to the 31-limit in all_ratios. 
print(type(all_ratios))
print(len(all_ratios))

In the skeleton csound file provided, we need these ratios as cpspch format. Here is the csound code that converts a note in the 213 per octave scale into a fraction of an octave, that csound can turn into a frequency that can be performed.

<code>
ipitch table p5, 3 ; convert note number 0-213 (or whatever the root is) to oct.fract format in a table of values
kcps = cpspch(ioct + ipitch) ; convert oct.fract to Hz at krate 
</code>



<p>To build that table, we need these ratios onverted into cents of the ratio, then divide that by 10,000. The formula for converting a ratio into cents is used in the next cell.

In [ ]:
# conversion between ratios (just) and cents (1200 equal per octave)
# ratios to cents:
ratio = 51/50
print(f'convert the ratio {str(Fraction(ratio).limit_denominator(max_denominator = 100))} to cents: {round(1200 * np.log(ratio)/np.log(2),1)}')
# cents to ratio
cents = 34.3
print(f'convert {cents} cents to a ratio: {str(Fraction(np.power(2, cents/1200)).limit_denominator(max_denominator = 100))}')

In [ ]:
all_ratios.append(2) # we will need a ratio of 2/1 at the end of the list, so I append it here
# let's take a look at the list of ratios, converted into ratios we can see:
# first five ratios
print(f'first five ratios: {[str(Fraction(ratio).limit_denominator(max_denominator = 100)) for ratio in all_ratios[:5]] }')
# last five ratios
print(f'last five ratios: {[str(Fraction(ratio).limit_denominator(max_denominator = 100)) for ratio in all_ratios[-5:]] }')

In [ ]:
print(f'first five fractions that csound needs: {[1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios[:5]]}')
print(f'last five fractions that csound needs: {[1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios[-5:]]}')

In [ ]:
# we don't need the 0.0 at the front, so we load all the others into a numpy array
f3_array = np.array([1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios[1:]]) # convert it into a numpy array

# to load a function table, we have to give csound some information about it in a preface:
#                      +-- the function table number we are creating (3)
#                      |  +-- 0 start immediately
#                      |  |  +-- the size of the array (prefers a power of two)
#                      |  |  |     +-- the type of table format, in this case values that will serve as a lookup table indexed by the note number
#                      |  |  |     |   that returns an octave fraction to use to build a frequency in Hertz. negative to supress normalization
#                      |  |  |     |     
f3_preface = np.array([3, 0, 256, -2])

f3_array_ready_to_load = np.concatenate((f3_preface,f3_array)) 
# now it is ready to pass to csound. 

In [ ]:
print(f'first 5 values {[fract for fract in f3_array_ready_to_load[:5]]}') # should be [3.0, 0.0, 256.0, -2.0, 0.00549644275357497]
print(f'last 3 values {[fract for fract in f3_array_ready_to_load[-3:]]}') # should be [0.11432331422659722, 0.11450355724642503, 0.12]

In [ ]:
# pp.pprint(cassandra.T)

In [ ]:
# this can be used to load the dictionaries for every key in the diamond.

def remove_spaces(string):
    return string.replace(" ", "")
key_list = []
print('oton ranks A, B, C, & D')
r = 0
for row in cassandra:
    # if r % 2 == 0:
    n = 0
    notes = []
    ratio = remove_spaces(row[0][0:5])
    # print(f'\"{ratio}\"', end='\t')
    notes.append(ratio)
    for note in row:
        if n % 2  == 1:
            # print(f'{r}, {n}, {note}', end = '\t')
            notes.append(int(note))
        n += 1
    print(notes)
    key_list.append(notes)
    r += 1

print(f'uton ranks A, B, C, D ')    
r = 0
for row in cassandra.T:
    if r % 2 == 0:
        notes = []
        ratio = remove_spaces(row[0][0:5])
        notes.append(ratio)
    if r % 2 == 1:
        n = 0
        for note in row:
            notes.append(int(note))
            n += 1
        print(f'{notes}')
        key_list.append(notes)
    r += 1    

In [ ]:
# build the chord structures for the different modes for the 16 keys
# build the 16 tone scales in each of the keys in the diamond. otonal first, then utonal
# nested dictionary mode ('oton', 'uton'), key (ratio_name)
# I need to make these by rank also
# mode, rank, ratio
keys = {'oton': {key_list[0][0]: np.array(key_list[0][1:]),
                 key_list[1][0]: np.array(key_list[1][1:]),
                 key_list[2][0]: np.array(key_list[2][1:]),
                 key_list[3][0]: np.array(key_list[3][1:]),
                 key_list[4][0]: np.array(key_list[4][1:]),
                 key_list[5][0]: np.array(key_list[5][1:]),
                 key_list[6][0]: np.array(key_list[6][1:]),
                 key_list[7][0]: np.array(key_list[7][1:]),
                 key_list[8][0]: np.array(key_list[8][1:]),
                 key_list[9][0]: np.array(key_list[9][1:]),
                 key_list[10][0]: np.array(key_list[10][1:]),
                 key_list[11][0]: np.array(key_list[11][1:]),
                 key_list[12][0]: np.array(key_list[12][1:]),
                 key_list[13][0]: np.array(key_list[13][1:]),
                 key_list[14][0]: np.array(key_list[14][1:]),
                 key_list[15][0]: np.array(key_list[15][1:])},
        'uton': {key_list[16][0]: np.array(key_list[16][1:]),
                 key_list[17][0]: np.array(key_list[17][1:]),
                 key_list[18][0]: np.array(key_list[18][1:]),
                 key_list[19][0]: np.array(key_list[19][1:]),
                 key_list[20][0]: np.array(key_list[20][1:]),
                 key_list[21][0]: np.array(key_list[21][1:]),
                 key_list[22][0]: np.array(key_list[22][1:]),
                 key_list[23][0]: np.array(key_list[23][1:]),
                 key_list[24][0]: np.array(key_list[24][1:]),
                 key_list[25][0]: np.array(key_list[25][1:]),
                 key_list[26][0]: np.array(key_list[26][1:]),
                 key_list[27][0]: np.array(key_list[27][1:]),
                 key_list[28][0]: np.array(key_list[28][1:]),
                 key_list[29][0]: np.array(key_list[29][1:]),
                 key_list[30][0]: np.array(key_list[30][1:]),
                 key_list[31][0]: np.array(key_list[31][1:])}
                }
pp.pprint(keys['oton']['1/1'])
print(f'{keys.keys() = }')
for mode in keys.keys():
    for ratio in keys[mode]:
        print(f'{mode = }, {ratio = }, {keys[mode][ratio] = }')
        if first_item_only: break    

# check if a key is available in the mode before accessing it:
if '16/9' in keys['oton']: print(f'{keys["oton"]["16/9"] = }')

In [ ]:
# build the 8 note scales for each of the rank A, B, C, D otonal and utonal out of the 16 note scales
#                                                        start, end, step size
# make this dictionary nested: rank, mode
scales = {'A': {'oton': np.array([note % 16 for note in np.arange(0, 16, 2)]),
                'uton': np.array([note % 16 for note in np.arange(8, -8, -2)])}, # utonal goes down, so that the scale will go up.
          'B': {'oton': np.array([note % 16 for note in np.arange(2, 18, 2)]),
                'uton': np.array([note % 16 for note in np.arange(14, -2, -2)])},
          'C': {'oton': np.array([note % 16 for note in np.arange(1, 17, 2)]),
                'uton': np.array([note % 16 for note in np.arange(13, -2, -2)])},
          'D': {'oton': np.array([note % 16 for note in np.arange(3, 19, 2)]),
                'uton': np.array([note % 16 for note in np.arange(15, 0, -2)])}
         }
pp.pprint(scales)
print()
print(f'{scales.keys() = }')
for rank in scales.keys():
    print(f'{rank = }')
    for mode in scales[rank]:
        print(f'{mode = }, {scales[rank][mode] = }')  
    if first_item_only: break

In [ ]:
# build the masks to make all the scales start at the lowest note and end at the highest for each rank
# scale_mask[rank, mode, key]
# def build_scales(mode, key, rank):
# transforming this from letter keys to ratios. 

scale_mask = {'A': {'oton': {'1/1'  : np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                            '16/9'  : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '8/5'  : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '16/11': np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '4/3'  : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '16/13': np.array([1, 1, 1, 1, 1, 0, 0, 0]),
                             '8/7'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             '16/15': np.array([1, 1, 1, 1, 1, 1, 1, 0])},
                             
                    'uton': {'1/1'  : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '9/8'  : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '5/4'  : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '11/8' : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '3/2'  : np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                             '13/8' : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '7/4'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             '15/8' : np.array([1, 1, 1, 1, 1, 0, 0, 0])}
                   },
              'B': {'oton': {'1/1'  : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '16/9' : np.array([0, 0, 0, 0, 0, 0, 0, 0]), 
                             '8/5'  : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '16/11': np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '4/3'  : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '16/13': np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '8/7'  : np.array([1, 1, 1, 1, 1, 0, 0, 0]), 
                             '16/15': np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             },
                    'uton': {'1/1'  : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '9/8'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             '5/4'  : np.array([1, 1, 1, 1, 1, 0, 0, 0]),
                             '11/8' : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '3/2'  : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '13/8' : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '7/4'  : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '15/8' : np.array([0, 0, 0, 0, 0, 0, 0, 0])},
                   },
              'C': {'oton': {'1/1'  : np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                             '16/9' : np.array([1, 0, 0, 0, 0, 0, 0, 0]), 
                             '8/5'  : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '16/11': np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '4/3'  : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '16/13': np.array([1, 1, 1, 1, 1, 0, 0, 0]),
                             '8/7'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]), 
                             '16/15': np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             },
                    'uton': {'1/1'  : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '9/8'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             '5/4'  : np.array([1, 1, 1, 1, 1, 0, 0, 0]),
                             '11/8' : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '3/2'  : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '13/8' : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '7/4'  : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '15/8' : np.array([0, 0, 0, 0, 0, 0, 0, 0])},
                   },
              'D': {'oton': {'1/1'  : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '16/9' : np.array([0, 0, 0, 0, 0, 0, 0, 0]), 
                             '8/5'  : np.array([1, 0, 0, 0, 0, 0, 0, 0]),
                             '16/11': np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '4/3'  : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '16/13': np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '8/7'  : np.array([1, 1, 1, 1, 1, 0, 0, 0]), 
                             '16/15': np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             },
                    'uton': {'1/1'  : np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                             '9/8'  : np.array([1, 1, 1, 1, 1, 1, 1, 0]),
                             '5/4'  : np.array([1, 1, 1, 1, 1, 1, 0, 0]),
                             '11/8' : np.array([1, 1, 1, 1, 1, 0, 0, 0]),
                             '3/2'  : np.array([1, 1, 1, 1, 0, 0, 0, 0]),
                             '13/8' : np.array([1, 1, 1, 0, 0, 0, 0, 0]),
                             '7/4'  : np.array([1, 1, 0, 0, 0, 0, 0, 0]),
                             '15/8' : np.array([1, 0, 0, 0, 0, 0, 0, 0])},
                   }
             }

In [ ]:
pp.pprint(scale_mask['A'])
# print()
print(f'{scale_mask.keys() = }')
for rank in scale_mask.keys():
    for mode in scale_mask[rank]:
        print(f'{rank = }, {mode = }', end = ' ')
        for ratio in scale_mask[rank][mode]:
            print(f'{ratio = }', end = ' ')
            print(f'{scale_mask[rank][mode][ratio] = }')
    if first_item_only: break

In [ ]:
# choose the notes in scale for each rank (A, B, C, D), mode (oton, uton), & inversion (1, 2, 3, 4)
# this took a while to figure out how to construct a triple nested dictionary
# access the four note chords by specifying inversions[rank][mode][inv]
# where rank is in (A, B, C, D) mode is in (oton, uton), and inv is in (1, 2, 3, 4)
# use this as the index into a keys['oton']['16/9']

inversions = {'A': {'oton': {1: np.array([0, 4, 8, 12]),
                             2: np.array([4, 8, 12, 0]),
                             3: np.array([8, 12, 0, 4]),
                             4: np.array([12, 0, 4, 8])},
                    'uton': {1: np.flip(np.array([0, 4, 8, 12])),
                             2: np.flip(np.array([4, 8, 12, 0])),
                             3: np.flip(np.array([8, 12, 0, 4])),
                             4: np.flip(np.array([12, 0, 4, 8]))}}, 
              'B': {'oton': {1: np.array([2, 6, 10, 14]),
                             2: np.array([6, 10, 14, 2]),
                             3: np.array([10, 14, 2, 6]),
                             4: np.array([14, 2, 6, 10])
                            },
                    'uton': {1: np.flip(np.array([2, 6, 10, 14])),
                             2: np.flip(np.array([6, 10, 14, 2])),
                             3: np.flip(np.array([10, 14, 2, 6])),
                             4: np.flip(np.array([14, 2, 6, 10]))}}, 
              'C': {'oton': {1: np.array([1, 5, 9, 13]),
                             2: np.array([5, 9, 13, 1]),
                             3: np.array([9, 13, 1, 5]),
                             4: np.array([13, 1, 5, 9])},
                    'uton': {1: np.flip(np.array([2, 6, 10, 14])),
                             2: np.flip(np.array([6, 10, 14, 2])),
                             3: np.flip(np.array([10, 14, 2, 6])),
                             4: np.flip(np.array([14, 2, 6, 10]))}}, 
              'D': {'oton': {1: np.array([3, 7, 11, 15]),
                             2: np.array([7, 11, 15, 3]),
                             3: np.array([11, 15, 3, 7]),
                             4: np.array([15, 3, 7, 11])},
                    'uton': {1: np.flip(np.array([3, 7, 11, 15])),
                             2: np.flip(np.array([7, 11, 15, 3])),
                             3: np.flip(np.array([11, 15, 3, 7])),
                             4: np.flip(np.array([15, 3, 7, 11]))}
                   }
             }

pp.pprint(inversions)
print()
print(f'{inversions.keys() = }')
for rank in inversions.keys():
    print(f'{rank = }')
    # print(f'{rank = }: {[inversions[rank][mode] for mode in inversions[rank].keys()]}')
    for mode in inversions[rank]:
        print(f'{mode = }')
        for chord_inversion in inversions[rank][mode]:
            print(f'inversion: {chord_inversion}: {inversions[rank][mode][chord_inversion] = }')
    if first_item_only: break

In [ ]:
# for each of the 8 keys, build the chords from the scales created above.
# key is a string index into the keys dictionary: e.g. mode, ratio
# inversion is the string index into the inversions dictionary e.g. 'A_oton_1'
# the inversion includes the rank is which of the four tetrachords in each scale, A, B, C, or D
# And it specifies which note is on top
# in

def build_chords(mode, ratio, rank, inversion):
    # .reshape(-1,1)  is needed for the Balloon_Drum_Music_Scales.ipynb or it screws up the dimensions
    chord = np.array([keys[mode][ratio][note] for note in (inversions[rank][mode][inversion])]) 
    return chord


print(f'{[build_chords("oton", "16/9","A", chord_inversion) for chord_inversion in inversions["A"]["oton"]]}')

In [ ]:
def build_scales(mode, ratio, rank):
    # print(f'{key = }, {mode = }, {rank = }')
    scale = np.array([keys[mode][ratio][note] for note in (scales[rank][mode])])
    return scale

ratio = '16/9'
mode = 'oton'

print(f'{[("rank:", rank, build_scales(mode, ratio, rank)) for rank in scales.keys()]}')

In [ ]:
# dictionary of ratios used in small steps in scales in ranks A & B.
# The key is a duple of steps, the items is the ratio
# do I need the utonal ratios? No, the ratios are the same. 
# these are missing ranks C & D, which will need to be remedied. I ran out of room in the csound csd file. Relocation required.
ratio_table = {(0,2): 9/8, (0,4): 5/4, (1,3): 19/17, (1,5): 21/17, (2,4): 10/9, (2,6): 11/9, (3,5): 21/19, (3,7): 23/19,
          (4,6): 11/10, (4,8): 6/5, (5,7): 23/21, (5,9): 25/21, (6,8): 12/11, (6,10): 13/11, (7,9): 25/23, (7,11): 27/23,
          (8,10): 13/12, (8,12): 7/6, (9,11): 27/25, (9,13): 25/21, (10,12): 14/13, (10,14): 15/13, (11,13): 29/27, (11,15): 31/27,
          (12,14): 15/14, (12,0): 8/7, (13,15): 31/29, (13,1): 34/29, (14,0): 16/15, (14,2): 6/5, (15,1): 34/31, (15,3): 38/31,
          (2,0): 8/9, (4,0): 4/5, (3,1): 17/19, (5,1): 17/21, (4,2): 9/10, (6,2): 9/11, (5,3): 19/21, (7,3): 19/23, (6,4): 10/11, 
          (8,4): 5/6, (7,5): 21/23, (9,5): 21/25, (8,6): 11/12, (10,6): 11/13, (9,7): 23/25, (11,7): 23/27, (10,8): 12/13, 
          (12,8): 6/7, (11,9): 25/27, (13.9): 21/25, (12,10): 13/14, (14,10): 13/15, (13,11): 27/29, (15,11): 27/31,
          (14,12): 14/15, (0,12): 7/8, (15,13): 29/31, (1,13): 29/34, (0,14): 15/16, (2,14): 5/6, (1,15): 31/34, (3,15): 31/38}
# I can create this data structure programmatically and add them as I need to use them.

print()
print(f'unsorted ratio_table: {[str(Fraction(value).limit_denominator()) for key,value in ratio_table.items()]}') 
step_start = 2
step_stop = step_start + 4
print(f'{str(Fraction(ratio_table[(step_start, step_stop)]).limit_denominator()) = }')

In [ ]:
# create the ratio table automatically. 
# what will it need? the steps are the same for both otonal and utonal. 

In [ ]:
# dictionary of glissandos. Key is the type of frequency alteration (trill, slide), with each type of frequency alteration
# is a dictionary of key ratio, item is the csound function table that performs the specific alteration
gliss = {'trills_2_step' : {16/15: 176, 15/14: 177, 14/13: 178, 13/12: 179, 12/11: 180, 11/10: 181, 10/9: 182, 9/8: 183, 8/7: 184, 15/13: 185,
                7/6: 186, 13/11: 187, 6/5: 188, 11/9: 189, 5/4: 190, 15/16: 191, 14/15: 192, 13/14: 193, 12/13: 194, 11/12: 195, 10/11: 196,
                9/10: 197, 8/9: 198, 7/8: 199, 13/15: 200, 6/7: 201, 11/13: 202, 5/6: 203, 9/11: 204, 4/5: 205},
         'trills_8_step': {16/15: 206, 15/14: 207, 14/13: 208, 13/12: 209, 12/11: 210, 11/10: 211, 10/9: 212, 9/8: 213, 8/7: 214, 15/13: 215,
                7/6: 216, 13/11: 217, 6/5:218, 11/9: 219, 5/4: 220, 15/16: 221, 14/15: 222, 13/14: 223, 12/13: 224, 11/12: 225, 10/11: 226,
                9/10: 227, 8/9: 228, 7/8: 229, 13/15: 300, 6/7: 301, 11/13: 302, 5/6: 303, 9/11: 304, 4/5: 305},
         'slide': {16/15: 14, 15/14: 15, 14/13: 16, 13/12: 17, 12/11: 18, 11/10: 19, 10/9: 20, 9/8: 21, 8/7: 22, 15/13: 23,
                7/6: 24, 13/11: 25, 6/5: 26, 11/9: 27, 5/4: 28, 15/16: 29, 14/15: 30, 13/14: 31, 12/13: 32, 11/12: 33, 10/11: 34,
                9/10: 35, 8/9: 36, 7/8: 37, 13/15: 38, 6/7: 39, 11/13: 40, 5/6: 41, 9/11: 42, 4/5: 43}
        }
print(f'{gliss.keys() = }')
for gliss_type in gliss.keys():
    print(f'{gliss_type = }')
    print(f'{[((str(Fraction(ratio_float).limit_denominator())), g_value) for ratio_float, g_value in gliss[gliss_type].items()]}')
    if first_item_only: break

In [ ]:
o_numerator = np.arange(16,32,1)
u_denominator = np.arange(16,32,1)

print(f'list of fractions in o_numerator over 16: {[str(Fraction(num/16).limit_denominator()) for num in o_numerator]}')
print(f'list of fractions in 16 over u_denominator: {[str(Fraction(32/num).limit_denominator()) for num in u_denominator]}')

In [ ]:
print(f'slide, gliss up. utonal')
for degree in np.arange(16, 0, -2):
    key_to_ratio = ((degree % 16), (degree - 2) % 16)
    print(f'steps: {key_to_ratio}: ratio: {Fraction(ratio_table[key_to_ratio]).limit_denominator()}')
    print(f'2 step trill ftable: {gliss["trills_2_step"][ratio_table[key_to_ratio]]}')
    print(f'8 step trill ftable: {gliss["trills_8_step"][ratio_table[key_to_ratio]]}')
    print(f'slide ftable: {gliss["slide"][ratio_table[key_to_ratio]]}')   
    key_to_ratio = ((degree % 16), (degree - 4) % 16)
    print(f'steps: {key_to_ratio}: ratio: {Fraction(ratio_table[key_to_ratio]).limit_denominator()}')
    print(f'2 step trill ftable: {gliss["trills_2_step"][ratio_table[key_to_ratio]]}')
    print(f'8 step trill ftable: {gliss["trills_8_step"][ratio_table[key_to_ratio]]}')
    print(f'slide ftable: {gliss["slide"][ratio_table[key_to_ratio]]}')  
    if first_item_only: break

In [ ]:
print(f'go down')    
for degree in np.arange(0, 16, 2):
    key_to_ratio = ((degree + 2) % 16, degree)
    print(f'steps: {key_to_ratio}: ratio: {Fraction(ratio_table[key_to_ratio]).limit_denominator()}')
    print(f'2 step trill ftable: {gliss["trills_2_step"][ratio_table[key_to_ratio]]}')
    print(f'8 step trill ftable: {gliss["trills_8_step"][ratio_table[key_to_ratio]]}')
    print(f'slide ftable: {gliss["slide"][ratio_table[key_to_ratio]]}')                  
    key_to_ratio = ((degree + 4) % 16, degree)
    print(f'steps: {key_to_ratio}: ratio: {Fraction(ratio_table[key_to_ratio]).limit_denominator()}')
    print(f'2 step trill ftable: {gliss["trills_2_step"][ratio_table[key_to_ratio]]}')
    print(f'8 step trill ftable: {gliss["trills_8_step"][ratio_table[key_to_ratio]]}')
    print(f'slide ftable: {gliss["slide"][ratio_table[key_to_ratio]]}')       
    if first_item_only: break

In [ ]:
# Build the data structures required for the csound processing
# voices to use in the orchestra:
# finger piano 1, bass fp 2, balloon drum low 3, med 4, high 5, bass flute 6, oboe 7, clarinet 8, bassoon 9, french horn 10
# 1    2   3   4    5   6   7   8    9    10
# fnp bfp bdl bdm  bdh bflt obo clar bass frn
#                 S   A   T   B   S   A   T   B   S   A   T   B
voices = 16 # how many voices for all the chorales - I've only tested 16, 32,& 64.
#        bass flute obo clar bsn frn 
list_voices =    [6,  7,  8,  9, 10,  6,  6,  6,  6,  6,  6,  6]
list_velocity = [66, 65, 64, 64, 65, 66, 64, 65, 67, 66, 66, 65]
o_vol = 23 # overall volume

list_volume = [o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol, o_vol]
list_holds = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
list_envs = [1, 0, 17, 16] 
list_envs = [list_envs, list_envs, list_envs, list_envs, list_envs, list_envs, 
             list_envs, list_envs, list_envs, list_envs, list_envs, list_envs]
low_p = 0.25 #  probability choice - rarely choose to hold the note, most events are stacatto
list_env_probs = [1 - (low_p * 3), low_p, low_p, low_p]
print(f'{list_env_probs = }')
all_env_probs = [list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, 
                  list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs, list_env_probs]
#                
voice_array = (list_voices, list_holds, list_volume, list_velocity, list_envs, all_env_probs)
i = 0
for variable in voice_array:
    assert len(variable) == len(voice_array[0])
    i += 1
for env, probs in zip(list_envs, all_env_probs):
    assert len(env) == len(probs)
npt.assert_almost_equal(np.sum(list_env_probs), 1.0,0.001)

In [ ]:
concat_chorale = np.load('balloon_drum_percussion.npy')
print(concat_chorale.shape)

In [ ]:
###### build_scales("oton", "16/9", rank)
ratio = "16/9"
print(f'{keys["oton"].keys() = }')
print(f'{scales.keys() = }')
for rank in scales:
    for mode in scales[rank]:
        if '16/9' in keys[mode].items(): print(f'{rank = }, {mode = }, {build_scales(mode, ratio, rank) = }')      

In [ ]:
# confirm that all scales in all used keys, all ranks are all going from low to high, and distance from 0 to 7 less than OCTAVE_SIZE
lowest_travel = 255
highest_travel = 0
# iterate on scale_mask[rank, mode, key] rather than keys, since keys does not include a reference to rank
# first_item_only = False
# first_item_only = True
oct = 1
for rank in scale_mask: # 'oton' and 'uton'
    for mode in scale_mask[rank]: # 'A', 'B', etc.
        for ratio in scale_mask[rank][mode]: # 1/1, 16/9, etc
            built_scale = build_scales(mode, ratio, rank)
            built_mask = scale_mask[rank][mode][ratio]
            after_sub = np.subtract((built_scale  + (oct * OCTAVE_SIZE)), built_mask * OCTAVE_SIZE)
            print(f'{rank = }, {mode = }, {ratio = }, {after_sub  = }')
            low = min(after_sub)
            high = max(after_sub)
            diff = high - low
            if diff < lowest_travel: lowest_travel = diff
            if diff > highest_travel: highest_travel = diff
            if diff > 217: 
                print(f'{low = }, {high = }, {diff = }')
                print(f'{built_scale}\n{built_mask}\n{built_scale  + (oct * OCTAVE_SIZE)}\n{built_mask * OCTAVE_SIZE}\n{after_sub}')
    if first_item_only: break
print(f'{lowest_travel = }, {highest_travel = }')

In [ ]:
# confirm that you have created chords properly
# def build_chords(mode, ratio, rank, inversion):
# def build_chords(mode, ratio, rank, inversion):
oct = 1
for rank in inversions: # rank A, B, C, D
    # print(f'{rank = }')
    for mode in keys: # 'oton', 'uton'
        # print(f'{mode = }')
        for ratio in keys[mode]: # '1/1', '32/17', '16/9', etc
            # print(f'{ratio = }')
            for inv in inversions[rank][mode]: # 1, 2, 3, 4
                # print(f'{inv = }')
                print(f'{rank = }, {mode = }, {ratio = }, {inv = }, : {build_chords(mode, ratio, rank, inv) + (oct * OCTAVE_SIZE)}')
                # if first_item_only: break
            if first_item_only: break
    if first_item_only: break


In [ ]:
# pick 3 random note orders
for i in np.arange(3):
    print(f'{rng.choice(([1,2,3,4,5,6,7,8]), size = 8, replace = False, shuffle = True)}')

In [ ]:
stretch.start_logging({'log': LOGNAME})
csd_content, lines = p.load_csd(CSD_FILE)
p.logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
cs, pt = p.load_csound(csd_content)

In [ ]:
oct = 3
rank = 'A'
inversion = 1
ratio = '16/9'
mode = 'oton'
note_count = 4 # 8 for scale
flute = np.subtract(build_scales(mode, ratio, rank) + (oct * OCTAVE_SIZE), scale_mask[rank][mode][ratio] * OCTAVE_SIZE) # shape = (8,) 

# flute = build_chords(key, mode, rank, inversion) + (oct * OCTAVE_SIZE) # shape = (4,)
print(f'After creating flute array: {flute.shape = }, {flute = }')
last_note = np.array((flute[0] + OCTAVE_SIZE,), dtype = int) # shape = (1,) 
print(f'{last_note.shape = }, {last_note = }')
# # shuffle the notes
# flute = rng.choice(flute, size = note_count, replace = False, shuffle = True)
pad1 = np.zeros((1,), dtype = int)
print(f'{pad1.shape = }, {pad1 = }') # shape = (1,)
flute = np.concatenate((flute, last_note, pad1), axis = 0) # all dimensions along the 0 axis match
print(f'After concatenating last note, and zero: {flute.shape = }, {flute = }')
flute = flute.reshape(-1,1).transpose()
print(f'reshape and transposed: {flute.shape = }, {flute = }')

challenging_steps = np.empty((0,4),dtype=int) # no challenging steps yet.

csound_params = {'piece': flute, 'challenging_steps': challenging_steps, 
                 'upsample': ([254, 255, 0, 1, 2]), 
                 'min_delay': 0, 'tpq': 1.0, 'log': LOGNAME, 
                 'csd_file': CSD_FILE, 'zfactor': 95, 'voice_array': voice_array, 'octave_size': OCTAVE_SIZE} 

pfields = stretch.piano_roll_to_pfields(csound_params) 
pfields.sort() 
for field in pfields:
    pt.scoreEvent(0, 'i', field)
    p.logging.info(f'{field = }')

In [ ]:
p.printMessages(cs)
# delay_time =  max(csound_params["min_delay"],len(pfields) // 50) # need enough time to prevent csound being told to stop 
# print(f'about to delay to allow ctcsound to process the notes. delay_time: {delay_time}')
last_start = pfields[-1][1]
print(f'last start time was at {round(last_start,1)}. Set f0 to {round(last_start+1,1)}')
# time.sleep(delay_time) # once you hit the next line csound stops
pt.stop() # this is important I think. It closes the output file.
pt.join()   
p.printMessages(cs)    
cs.reset()

In [ ]:
stretch.start_logging({'log': LOGNAME})
csd_content, lines = p.load_csd(CSD_FILE)
p.logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
cs, pt = p.load_csound(csd_content)

In [ ]:
# create a sliding chord from A to B
oct = 4
key = '16/9'
mode = 'oton'
rank = 'A'
inv = 2
rank_2 = 'B'
inv_2 = 1
print(f'{[(inversions[rank][mode][inv][note], inversions[rank_2][mode][inv_2][note]) for note in np.arange(4)]}')
print(f'{[(inversions[rank_2][mode][inv_2][note], inversions[rank][mode][inv][note]) for note in np.arange(4)]}')
print(f'{[str(Fraction(ratio_table[(inversions[rank][mode][inv][note], inversions[rank_2][mode][inv_2][note])]).limit_denominator()) for note in np.arange(4)]}')
print(f'{[str(Fraction(ratio_table[(inversions[rank_2][mode][inv_2][note], inversions[rank][mode][inv][note])]).limit_denominator()) for note in np.arange(4)]}')

slides_down = [gliss["slide"][ratio_table[(inversions[rank][mode][inv][note], inversions[rank_2][mode][inv_2][note])]] for note in np.arange(4)]
slides_up = [gliss["slide"][ratio_table[(inversions[rank_2][mode][inv_2][note], inversions[rank][mode][inv][note])]] for note in np.arange(4)]
slides = np.concatenate((slides_down, slides_up))
print(f'{np.array(slides_up).shape = }')
trill_2_down = [gliss["trills_2_step"][ratio_table[(inversions[rank][mode][inv][note], inversions[rank_2][mode][inv_2][note])]] for note in np.arange(4)]
trill_2_up = [gliss["trills_2_step"][ratio_table[(inversions[rank_2][mode][inv_2][note], inversions[rank][mode][inv][note])]] for note in np.arange(4)]
trill_2 = np.concatenate((trill_2_down, trill_2_up), axis = 0)
print(f'{trill_2.shape = }')
trill_8_down = [gliss["trills_8_step"][ratio_table[(inversions[rank][mode][inv][note], inversions[rank_2][mode][inv_2][note])]] for note in np.arange(4)]
trill_8_up = [gliss["trills_8_step"][ratio_table[(inversions[rank_2][mode][inv_2][note], inversions[rank][mode][inv][note])]] for note in np.arange(4)]
trill_8 = np.concatenate((trill_8_down, trill_8_up), axis = 0)
print(f'{trill_8.shape = }')
print(f'{slides_down = }')

In [ ]:
# create a sliding chord from A to B
# build_chords(mode, ratio, rank, inversion):
oct = 4
ratio = '16/9'
mode = 'oton'

# first chord on rank 'A' inversion 2
rank = 'A'
inv = 2
print(f'{ratio = }, {rank = }, {mode = }, {inv = }')
chord = build_chords(mode, ratio, rank, inv) 
print(f'After creating chord array: {chord.shape = }, {chord = }')
chord_plus_slide = np.column_stack((np.array(slides_up), chord))
print(f'{chord_plus_slide.shape = }, {chord_plus_slide = }')

In [ ]:
# second chord on rank 'B' inversion 1
rank_2 = 'B'
inv_2 = 1
print(f'{key = }, {rank_2 = }, {mode = }, {inv_2 = }')
print(f'{oct * OCTAVE_SIZE = }')
chord2 = build_chords(mode, ratio, rank_2, inv_2)
print(f'After creating chord2 array: {chord2.shape = }, {chord2 = }')

chords = np.column_stack((chord, chord2))
chords += (oct * OCTAVE_SIZE)
print(f'After concatenating chords: {chords.shape = }, {chords = }')

In [ ]:
pad1 = np.zeros((4,1), dtype = int)
print(f'{pad1.shape = }, {pad1 = }') # shape = (1,)

chords = np.concatenate((chords, pad1), axis = 1) 
print(f'After concatenating chords plus pad1: {chords.shape = }, {chords = }')
chords = np.repeat(chords, 16, axis = 1)
print(f'repeat each note 16 times: {chords.shape = }')

In [ ]:
challenging_steps = np.empty((0,4),dtype=int) # no challenging steps yet.

csound_params = {'piece': chords, 'challenging_steps': challenging_steps, 
                 'upsample': ([254, 255, 0, 1, 2]), 
                 'min_delay': 0, 'tpq': 1.0, 'log': LOGNAME, 
                 'csd_file': CSD_FILE, 'zfactor': 95, 'voice_array': voice_array, 'octave_size': OCTAVE_SIZE} 

pfields = stretch.piano_roll_to_pfields(csound_params) 
# this is where I discovered that the slides were not assigned to the correct notes. 
for i in range(8):
    pfields[i][9] = slides[i] # slide down 9/10 35 or trill down 2 steps 197 - 8 steps (528) 227 
    print(f'note: {pfields[i][4]} slide: {slides[i]}')
    
    # set the upsample to compensate for the direction of the slope to keep the vibrato roughly constant        
pfields[0][10] = 255
pfields[1][10] = 255
pfields[2][10] = 255
pfields[3][10] = 255

pfields[4][10] = 1
pfields[5][10] = 1
pfields[6][10] = 1
pfields[7][10] = 1

for i in range(len(pfields)):
    pfields[i][6] = 6 # all bass flute

print(f'inst\tstar\tdur\tvel\tton\toct\tvoi\tste\tEnv\tGli\tups\tREn\t2gl\t3gj\tvol\tchn\tden\tt_s')
print(f'0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\t13\t14\t15\t16\t17')
print(f'------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------')

for row in pfields:
    for field in row:
        print(f'{round(field,1)}',end='\t')

for field in pfields:
    pt.scoreEvent(0, 'i', field)
    p.logging.info(f'{field = }')

In [ ]:
p.printMessages(cs)
# delay_time =  max(csound_params["min_delay"],len(pfields) // 50) # need enough time to prevent csound being told to stop 
# print(f'about to delay to allow ctcsound to process the notes. delay_time: {delay_time}')
last_start = pfields[-1][1]
print(f'last start time was at {round(last_start,1)}. Set f0 to {round(last_start+1,1)}')
# time.sleep(delay_time) # once you hit the next line csound stops
pt.stop() # this is important I think. It closes the output file.
pt.join()   
p.printMessages(cs)    
cs.reset()

In [ ]:
# Explore the pfields using pandas
pfield_columns = ['instrument', 'start_time', 'duration', 'velocity', 'tone', 'octave','voice', 'stereo', 'envelope',
                                            'gliss', 'upsample', 'renv', '2gliss', '3gliss', 'volume', 'channel', 'density', 'time_step']
pfields_df = pd.DataFrame(pfields, columns = pfield_columns)

In [ ]:
display(pfields_df.head(50))

In [ ]:
pfields_df.describe()

In [ ]:
# list the value_counts for one column 
# important_columns = ['velocity','tone','octave', 'voice', 'envelope', 'upsample', 'volume', 'channel', 'density']
important_columns = ['gliss']
print(f'value_counts for {important_columns = }')
for column in pfield_columns:
    if column in important_columns: 
        print(f'{column = }')
        print(f'{pfields_df[column].value_counts()}')

In [ ]:
time_step_headings = ['instrument','duration', 'hold', 'velocity', 'tone', 'octave', 
             'voice', 'stereo', 'l_envelope', 'gliss1', 'upsample', 
             'r_envelope', 'gliss2', 'gliss3', 'volume']
print(f'{len(time_step_headings) = }')

In [ ]:
# create a sliding chord from A to B
oct = 4
key = '16/9'
mode = 'oton'
rank = 'A'
inv = 2
print(f'{key = }, {rank = }, {mode = }, {inv = }')
chord = build_chords(mode, ratio, rank, inv) 
print(f'After creating chord array: {chord.shape = }, {chord = }')
print(f'{slides_down = }')
# second chord on rank 'B' inversion 1
rank_2 = 'B'
inv_2 = 1
print(f'{key = }, {rank_2 = }, {mode = }, {inv_2 = }')
chord2 = build_chords(mode, ratio, rank_2, inv_2)
print(f'After creating chord2 array: {chord2.shape = }, {chord2 = }')
print(f'{slides_up = }')

In [ ]:
# don't run this twice in a row, it causes panic.
stretch.start_logging({'log': LOGNAME})
csd_content, lines = p.load_csd(CSD_FILE)
p.logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
cs, pt = p.load_csound(csd_content)

In [ ]:
time_steps = np.array([[1, 0, 5, 60, 34, 4, 6, 8, 1, 35, 255, 0, 0, 0, 30],
                        [1, 0, 5, 60, 90, 4, 6, 8, 1, 33, 255, 0, 0, 0, 30],
                        [1, 0, 5, 60, 138, 4, 6, 8, 1, 31, 255, 0, 0, 0, 30],
                        [1, 0, 5, 60, 179, 4, 6, 8, 1, 29, 255, 0, 0, 0, 30],
                        [1, 5, 5, 60, 0, 4, 6, 8, 1, 20, 1, 0, 0, 0, 30],
                        [1, 5, 5, 60, 63, 4, 6, 8, 1, 18, 1, 0, 0, 0, 30],
                        [1, 5, 5, 60, 115, 4, 6, 8, 1, 16, 1, 0, 0, 0, 30],
                        [1, 5, 5, 60, 159, 4, 6, 8, 1, 14, 1, 0, 0, 0, 30]
                     ])

print(f'{[column for column in time_steps.T]}')

In [ ]:
for row in time_steps:
    pt.scoreEvent(0, 'i', row)
    print(f'{row = }')

In [ ]:
p.printMessages(cs)
# delay_time =  max(csound_params["min_delay"],len(pfields) // 50) # need enough time to prevent csound being told to stop 
# print(f'about to delay to allow ctcsound to process the notes. delay_time: {delay_time}')
last_start = pfields[-1][1]
print(f'last start time was at {round(last_start,1)}. Set f0 to {round(last_start+1,1)}')
# time.sleep(delay_time) # once you hit the next line csound stops
pt.stop() # this is important I think. It closes the output file.
pt.join()   
p.printMessages(cs)    
cs.reset()

In [ ]:
voice_time = {'flute': {'start_time': 0,
                   'voice_number': 6},
         'oboe': {'start_time': 0, 
                  'voice_number': 7}
        }

pp.pprint(voice_time)

In [ ]:
print(f'{time_step_headings = }')

In [ ]:
# don't run this twice in a row, it causes panic.
stretch.start_logging({'log': LOGNAME})
csd_content, lines = p.load_csd(CSD_FILE)
p.logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
cs, pt = p.load_csound(csd_content)

In [ ]:
duration = np.array([0, 0, 0, 5.2, 0, 0, 0, 5.2], dtype = int)
hold = 5
velocity = 60
upsamples = np.array([255,1])
volume = 30

time_step_dict = {'instrument': np.ones((8,), dtype = int),
                  'duration': duration,
                  'hold': np.ones((8,), dtype = int) * hold,
                  'velocity': np.ones((8,), dtype = int) * velocity,
                  'tone': np.concatenate((chord, chord2)),
                  'octave': np.ones((8,), dtype = int) * 4,
                  'voice': np.repeat(voice_time["flute"]["voice_number"],8),
                  'stereo': np.ones((8,), dtype = int) * 8, 
                  'l_envelope': np.ones((8,), dtype = int) * 1, 
                  'gliss1': slides, 
                  'upsample': np.concatenate((np.repeat(upsamples[0],4), np.repeat(upsamples[1],4))),
                  'r_envelope': np.ones((8,), dtype = int) * 1, 
                  'gliss2' : np.zeros((8,), dtype = int),
                  'gliss3': np.zeros((8,), dtype = int),
                  'volume': np.ones((8,), dtype = int) * volume}
pp.pprint(time_step_dict)

In [ ]:
duration = np.array([0, 0, 0, 5.2, 0, 0, 0, 5.2], dtype = int)
hold = 5
velocity = 60
upsamples = np.array([255,1])
volume = 30

time_step_dict2 = {'instrument': np.ones((8,), dtype = int),
                  'duration': duration,
                  'hold': np.ones((8,), dtype = int) * hold,
                  'velocity': np.ones((8,), dtype = int) * velocity,
                  'tone': np.concatenate((chord, chord2)),
                  'octave': np.ones((8,), dtype = int) * 4,
                  'voice': np.repeat(voice_time["flute"]["voice_number"],8),
                  'stereo': np.ones((8,), dtype = int) * 8, 
                  'l_envelope': np.ones((8,), dtype = int) * 1, 
                  'gliss1': slides, 
                  'upsample': np.concatenate((np.repeat(upsamples[0],4), np.repeat(upsamples[1],4))),
                  'r_envelope': np.ones((8,), dtype = int) * 1, 
                  'gliss2' : np.zeros((8,), dtype = int),
                  'gliss3': np.zeros((8,), dtype = int),
                  'volume': np.ones((8,), dtype = int) * volume}

pp.pprint(time_step_dict2)

In [ ]:
print(f'{[len(time_step_dict[time_step]) == len(time_step_dict["instrument"]) for time_step in time_step_dict.keys()]}')
print(f'{time_step_dict.keys() = }')
print(f'{[time_step_dict[time_step] for time_step in time_step_dict.keys()]}')
print(f'{len(time_step_dict) = }')
print(f'{[len(time_step_dict2[time_step]) == len(time_step_dict2["instrument"]) for time_step in time_step_dict2.keys()]}')
print(f'{time_step_dict2.keys() = }')
print(f'{[time_step_dict2[time_step] for time_step in time_step_dict2.keys()]}')
print(f'{len(time_step_dict2) = }')

In [ ]:
def array_from_dict(time_step_dict):
    time_step_array = np.empty((len(time_step_dict),len(time_step_dict["instrument"])), dtype = int)
    assert all([len(time_step_dict[time_step]) == len(time_step_dict["instrument"]) for time_step in time_step_dict.keys()]),\
        "not all items are same quantity"
    inx = 0
    for column in time_step_dict:
        time_step_array[inx] = time_step_dict[column]
        inx += 1
    return (time_step_array.T)

In [ ]:
time_step1 = array_from_dict(time_step_dict) # it comes back transposed into (rows, notes) containing duration, and hold
print(f'{time_step1.shape = }')
time_step2 = array_from_dict(time_step_dict2)
print(f'{time_step2.shape = }')
time_step_array = np.concatenate((time_step1, time_step2), axis = 0)
print(f'{time_step_array.shape = }')

In [ ]:
# ['instrument', 'start', 'duration', 'hold', 'velocity', 'tone', 'octave', 'voice', 'stereo', 
# 'l_envelope', 'gliss1', 'upsample', 'r_envelope', 'gliss2', 'gliss3', 'volume', 'elapsed']
print(f'{voice_time["flute"]["voice_number"] = }')

temp_array = copy.deepcopy(time_step_array) # this is necessary because I mess with the start column in the time_step_array 

#       +-- f says this is a function 
#       | +-- function is called # 330
#       | |   +-- start at time 0
#       | |   | +-- table has 256 time steps
#       | |   | |   +--  means don't rescale, 7 means straight lines
#       | |   | |   |  +--  table value starts at 1, meaning no change to pitch at the start
#       | |   | |   |  | +-- take this much time to reach next table value
#       | |   | |   |  | |  +-- still at no change to pitch
#       | |   | |   |  | |  | +-- over the next 128 time steps gradually go down towards 0.9375
#       | |   | |   |  | |  | |   +-- reach this value
#       | |   | |   |  | |  | |   |       +-- stay here for this number of time steps
#       | |   | |   |  | |  | |   |       |   +-- end here
#       | |   | |   |  | |  | |   |       |   |          +-- this is the number to send to csound to select ftable number 330. 29 + 301 = 330
f330 = (330, 0, 256, -7, 1, 16, 1, 128, 0.93750, 112, 0.93750) # 29
f332 = (332, 0, 256, -7, 1, 16, 1, 128, 0.92857, 112, 0.92857) # 31 
f334 = (334, 0, 256, -7, 1, 16, 1, 128, 0.91667, 112, 0.91667) # 33 
f336 = (336, 0, 256, -7, 1, 16, 1, 128, 0.90000, 112, 0.90000) # 35 
# mess with these

# f330 = (330, 0, 256, -7, 1, 16, 1, 128, 0.093750, 112, 0.093750) # 29
# f332 = (332, 0, 256, -7, 1, 16, 1, 128, 0.092857, 112, 0.092857) # 31 
# f334 = (334, 0, 256, -7, 1, 16, 1, 128, 0.091667, 112, 0.091667) # 33 
# f336 = (336, 0, 256, -7, 1, 16, 1, 128, 0.090000, 112, 0.090000) # 35 


pt.scoreEvent(0, 'f', f330)
pt.scoreEvent(0, 'f', f332)
pt.scoreEvent(0, 'f', f334)
pt.scoreEvent(0, 'f', f336)

voice_time["flute"]["start_time"] = 0 # convert the durations into start_time["flute"] for when the note begins to play based on the sum of the prior durations.
# print (f'{temp_array.shape = }')
current_duration = 0
inx = 0
for row in temp_array:
    voice_time["flute"]["start_time"] += current_duration
    current_duration = row[1] # the 'start' column
    # print(f'row number: {inx = }, before {current_duration = }, {voice["flute"]["start_time"] = }', end = '\t')
    temp_array[inx,1] = voice_time["flute"]["start_time"]
    # print(f'after {row[1] = }')
    pt.scoreEvent(0, 'i', row)
    inx += 1
print(f'{voice_time["flute"]["start_time"] = }')    
print(f'inst\tstart\thold\tvel\tton\toct\tvoi\tste\tEnv\tGli\tups\tREn\t2gl\t3gj\tvol')
print(f'0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\t13\t14')
print(f'------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- -------')
for row in temp_array:
    for field in row:
        print(f'{round(field,2)}',end='\t')
    print()

In [ ]:
p.printMessages(cs)
# delay_time =  max(csound_params["min_delay"],len(pfields) // 50) # need enough time to prevent csound being told to stop 
# print(f'about to delay to allow ctcsound to process the notes. delay_time: {delay_time}')
last_start = pfields[-1][1]
print(f'last start time was at {round(last_start,1)}. Set f0 to {round(last_start+1,1)}')
# time.sleep(delay_time) # once you hit the next line csound stops
pt.stop() # this is important I think. It closes the output file.
pt.join()   
p.printMessages(cs)    
cs.reset()

## Size of the Tonality Diamond, where n is the limit:
<img src='http://ripnread.com/listen/EulerTotientFunction.svg'>
There are 
7 members to the 5-limit diamond, 
13 to the 7-limit diamond, 
19 to the 9-limit diamond, 
29 to the 11-limit diamond, 
41 to the 13-limit diamond, and 
49 to the 15-limit diamond; 
these suffice for most purposes. <-- the editorial comment that got me working


In [ ]:
# brute force method

for limit in (5, 7, 9, 11, 13, 15, 31):
    end_denom = limit + 1
    start_denom = (end_denom) // 2
    # print(f'{limit = }, {start_denom = }, {end_denom = }')
    o_numerator = np.arange(start_denom, end_denom,1)
    u_denominator = np.arange(start_denom, end_denom, 1)
    print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
    all_ratios = []
    for oton_root in u_denominator:
        print()
        for overtone in o_numerator:
            if overtone < oton_root: oton = overtone * 2
            else: oton = overtone
            print(f'{str(Fraction(oton / oton_root).limit_denominator())}', end = '\t')
            all_ratios.append(oton / oton_root)

    all_ratios = list(dict.fromkeys(all_ratios)) # eliminate duplicates
    print(f'\nThere are {len(all_ratios)} unique members in the {limit}-limit diamond\n\n')      

In [ ]:
print(f'{str(Fraction(max(all_ratios)).limit_denominator())}') # what's the largest ratio in the set
print(f'{str(Fraction(min(all_ratios)).limit_denominator())}') # what's the smallest ratio

In [ ]:
# conversion between ratios (just) and cents (1200 equal per octave)
# ratios to cents:
ratio = 3/2
print(f'convert the ratio {str(Fraction(ratio).limit_denominator(max_denominator = 100))} to cents: {round(1200 * np.log(ratio)/np.log(2),1)}')
# cents to ratio
cents = 702
print(f'convert {cents} cents to a ratio: {str(Fraction(np.power(2, cents/1200)).limit_denominator(max_denominator = 100))}')

In [ ]:
# these are the values for steps using csound cpspch conversion. These numbers are in cents / 10000
csound_steps = np.array([0.0016727, 0.0033617, 0.0054964, 0.0056767, 0.0058692, 0.0060751, 0.0062961, 0.0065337, 0.0067900, 0.0070672, 
0.0073681, 0.0076956, 0.0080537, 0.0084467, 0.0088801, 0.0093603, 0.0098955, 0.0104955, 0.0111731, 0.0115458, 
0.0119443, 0.0124712, 0.0128298, 0.0133248, 0.0138573, 0.0144353, 0.0150637, 0.0157493, 0.0159920, 0.0165004, 
0.0170424, 0.0173268, 0.0176210, 0.0182404, 0.0189050, 0.0192558, 0.0196198, 0.0203910, 0.0212253, 0.0216687, 
0.0221309, 0.0241174, 0.0249171, 0.0241961, 0.0247741, 0.0253805, 0.0256950, 0.0258874, 0.0266871, 0.0275378, 
0.0277591, 0.0281358, 0.0289210, 0.0294135, 0.0297513, 0.0301847, 0.0304508, 0.0315641, 0.0327622, 0.0330761, 
0.0336130, 0.0340552, 0.0347408, 0.0352477, 0.0354547, 0.0359472, 0.0365825, 0.0369747, 0.0372408, 0.0374333, 
0.0386314, 0.0399090, 0.0401303, 0.0404442, 0.0409244, 0.0417508, 0.0424364, 0.0427373, 0.0435084, 0.0441278, 
0.0443081, 0.0446363, 0.0454214, 0.0459994, 0.0464428, 0.0467936, 0.0470781, 0.0475114, 0.0478259, 0.0498045, 
0.0516761, 0.0519551, 0.0524319, 0.0525745, 0.0528687, 0.0532428, 0.0536951, 0.0543015, 0.0551318, 0.0556737, 
0.0558796, 0.0563382, 0.0568717, 0.0571726, 0.0582512, 0.0591648, 0.0593718, 0.0597000, 0.0603000, 0.0606282, 
0.0608352, 0.0617488, 0.0628274, 0.0631283, 0.0636618, 0.0641204, 0.0643263, 0.0648682, 0.0656985, 0.0663049, 
0.0667672, 0.0671313, 0.0674255, 0.0676681, 0.0680449, 0.0683249, 0.0701955, 0.0721741, 0.0724886, 0.0729219, 
0.0732064, 0.0735572, 0.0740006, 0.0745786, 0.0753637, 0.0756919, 0.0758722, 0.0764916, 0.0772627, 0.0775636, 
0.0782492, 0.0790756, 0.0795558, 0.0798697, 0.0800910, 0.0813686, 0.0825667, 0.0827592, 0.0830253, 0.0834175, 
0.0840528, 0.0845453, 0.0847524, 0.0852592, 0.0859448, 0.0863870, 0.0869249, 0.0872478, 0.0884359, 0.0895492, 
0.0898153, 0.0902487, 0.0905865, 0.0910790, 0.0918642, 0.0922409, 0.0924622, 0.0933129, 0.0941126, 0.0943050, 
0.0946195, 0.0952259, 0.0958039, 0.0960829, 0.0968826, 0.0978691, 0.0983313, 0.0987747, 0.0996090, 0.1003802, 
0.1007442, 0.1010950, 0.1017596, 0.1024790, 0.1026732, 0.1029577, 0.1034996, 0.1040080, 0.1042507, 0.1049363, 
0.1055647, 0.1061427, 0.1066762, 0.1071702, 0.1076288, 0.1080557, 0.1084542, 0.1088269, 0.1095045, 0.1101045, 
0.1106397, 0.1111199, 0.1115533, 0.1119463, 0.1124044, 0.1126319, 0.1129328, 0.1132100, 0.1134663, 0.1137039, 
0.1139249, 0.1141308, 0.1143243, 0.1145036, 0.1166383, 0.1183273, 0.1200000])

In [ ]:
cent_array = csound_steps * 10_000
print(cent_array[:2]) # bottom two ratios in the 217 note per octave scale
print(cent_array[215:]) # top two ratios

In [ ]:
# convert from cpspch to ratios. start out with the csound value for cpspch
# show us the four values inserted in to the pitch table that were not in the tonality diamond.
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 200)) for step in cent_array[0:2]]}') 
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 200)) for step in cent_array[214:216]]}') 

In [ ]:
print(f'The ratios for the steps in the {len(cent_array)} step per octave scale:')
#                      +-- calculate the ratio by raising 2 to the power of (cents / 1200) 
#                      |                              +-- limit the denominator to a maximum of max_denominator
#                      |                              |                                                +-- array in csound * 10000
#                      |                              |                                                |
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 50)) for step in cent_array]}') 